# Parallel Programming with (Py)OpenCL for Fun and Profit

Gordon Inggs

Github: *Gordonei*

Day Job(s): 
* Science Data, City of Cape Town
* Application Acceleration Consultant, My Lounge/Back Garden*

\*weather permitting

## Talk Outline
* Why PyOpenCL (~5 mins)
* How to *x* with PyOpenCL (~15 mins)
  * Program CPUs, GPUs and more
  * Manipulate Memory
  * Do things ~~concurrently~~ in parallel
  
Github Repo: [Gordonei/pyconza-october-2018](https://github.com/Gordonei/pyconza-october-2018)

## Talk Approach
Use trusty vector sum example to illustrate points. Points should be a tight loop between imparting some information, then a demonstration of that information in action.

Encourage interruptions!

## Why PyOpenCL? ( $\approx$ Why Heterogeneous Compute? )
 
![Cat Lion Gif](./img/cat_lion.gif)

### Bad News: The Free Lunch is over

![Cat sad gif](./img/cat_sad.gif)

<img src="../img/freelunch_is_over.png" alt="Free Lunch is over graph" width="500"/>

*Source: Herb Sutter, The Free Lunch is Over*

### Good New: There are alternatives 

![Cat milk gif](./img/cat_milk.gif)

#### Performance-wise

![Double precision performance graph](./img/performance_graph.png)

*Source: Karl Rupp, ViennaCL*

#### Energy-wise

![Power performance graph](./img/power_graph.png)

*Source: Karl Rupp, ViennaCL*

#### Performance vs Energy

![Bitcoin mining performance graph](./img/performance_graph_II.png)

*Source: @oocBlog, The cost of securing the network*

# Sounds great, but...

![Cats on turntable](./img/cat_turntable.gif)

# The Challenges of Heterogeneous Computing
1. The Orientation Problem - turning things on!
2. The IO Problem - moving data around!
3. The Conceptual Problem - what am I doing?!?

PyOpenCL helps us overcome these challenges:
1. Program Multicore CPUs and GPUs. Also allows for device characteristics at runtime.
2. Allows for the efficient transfer of data. Exposes device memory hierachy.
3. Exposes task vs data parallelism.

# Programming Fancy Devices

In this section:
* What an OpenCL program is
* How to compile an OpenCL program
* How to run an OpenCL program

![Cat Metro Mayer gif](./img/cat_metro_mayer.gif)

What is OpenCL?

‘OpenCL (Open Computing Language) is the open, royalty-free standard
for cross-platform, parallel programming of diverse processors found in
personal computers, servers, mobile devices and embedded platforms.’

Intended for portable Heterogeneous computing, but not performance portability

2 APIs (programming and runtime) and a kernel language

## Khronos Group Promoters

![Khronos Group Promoters](./img/khronos_group_promoters.png)

## Selecting platforms &/ devices
Looking at what is available, then selecting the NVIDIA one.

In [1]:
import pyopencl
import numpy

In [2]:
ocl_platforms = (platform.name 
                 for platform in pyopencl.get_platforms())
print("\n".join(ocl_platforms))

Intel(R) OpenCL
Portable Computing Language
NVIDIA CUDA


In [3]:
nvidia_platform = [platform 
                   for platform in pyopencl.get_platforms() 
                   if platform.name == "NVIDIA CUDA"][0]
nvidia_devices = nvidia_platform.get_devices()

## OpenCL Programming Abstractions

![OpenCL Programming abstractions](./img/opencl_programming_abstractions.png)

## Building Programs
Using the PyOpenCL bindings to create an OpenCL context, then declaring OpenCL kernel code, and compiling it.

The code is for a simple vector sum, i.e. $\vec{c} = \vec{a} + \vec{b}$

In [4]:
nvidia_context = pyopencl.Context(devices=nvidia_devices)

program_source = """
      kernel void sum(global float *a, 
                      global float *b,
                      global float *c){
        int gid = get_global_id(0);
        c[gid] = a[gid] + b[gid];
      }
    """
nvidia_program_source = pyopencl.Program(nvidia_context, program_source)
nvidia_program = nvidia_program_source.build()

In [5]:
program_kernel_names = nvidia_program.get_info(pyopencl.program_info.KERNEL_NAMES)
print("Kernel Names: {}".format(program_kernel_names))

Kernel Names: sum


## OpenCL Runtime Abstractions

![OpenCL Runtime abstractions](./img/opencl_runtime_abstractions.png)

## Running Programs
Actually running the code using the Python bindings. 

Some memory and data needs to be setup first, but I'll elaborate on this later.

In [6]:
def run_ocl_kernel(queue, kernel, global_size,
                   input_tuples, output_tuples,
                   local_size = (32,)):
    
    # copying data onto the device
    for (array, buffer) in input_tuples:
        pyopencl.enqueue_copy(queue, src=array, dest=buffer)
    
    # running program on the device
    kernel_arguments  = [buffer for (_,buffer) in input_tuples] 
    kernel_arguments += [buffer for (_,buffer) in output_tuples]
        
    kernel(queue, global_size, local_size,
           *kernel_arguments)

    # copying data off the device
    for (arr, buffer) in output_tuples:
        pyopencl.enqueue_copy(queue, src=buffer, dest=arr)
        
    # waiting for everything to finish
    queue.finish()

# Trust, but verify:

In [7]:
def check_sum_results(a,b,c):
    c_ref = a + b
    err = numpy.abs(c - c_ref)
    if((err.sum() > 0.0).any()): 
        print("result does not match")
    else: 
        print("result matches!")   

In [8]:
# Synthetic data setup
N = int(2**20)
a = numpy.random.rand(N).astype(numpy.float32)
b = numpy.random.rand(N).astype(numpy.float32)
c = numpy.empty_like(a)

# Device Memory setup
a_nvidia_buffer = pyopencl.Buffer(nvidia_context,
                                  flags=pyopencl.mem_flags.READ_ONLY, 
                                  size=a.nbytes)
b_nvidia_buffer = pyopencl.Buffer(nvidia_context, 
                                  flags=pyopencl.mem_flags.READ_ONLY, 
                                  size=b.nbytes)
c_nvidia_buffer = pyopencl.Buffer(nvidia_context, 
                                  flags=pyopencl.mem_flags.WRITE_ONLY, 
                                  size=c.nbytes)

In [9]:
nvidia_queue = pyopencl.CommandQueue(nvidia_context)

input_tuples = ((a, a_nvidia_buffer), (b, b_nvidia_buffer), )
output_tuples = ((c, c_nvidia_buffer),)
run_ocl_kernel(nvidia_queue, nvidia_program.sum, (N,), input_tuples, output_tuples)

In [10]:
check_sum_results(a, b, c)

result matches!


In [11]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum, (N,), input_tuples, output_tuples)

2.65 ms ± 507 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


***Suggestion***: Characterise the OpenCL devices you can get access to, stepping up the size of $N$ by some factor (power of 2 is easy to reason about). 

# How to Manipulate Memory

In this section:
* Moving data between the host and OpenCL device
* Specifying storage on the device being used

![Cat box gif](./img/cat_box.gif)

## OpenCL Memory Abstractions

![OpenCL Memory](./img/opencl_memory_abstractions.png)

## Using global memory

In [12]:
def create_input_memory(context, input_arrays):
    return [(array, pyopencl.Buffer(context,
                                flags=pyopencl.mem_flags.READ_ONLY, 
                                size=array.nbytes))
        for array in input_arrays]

In [13]:
def create_output_memory(context, output_arrays):
    return [(array, pyopencl.Buffer(context,
                                flags=pyopencl.mem_flags.WRITE_ONLY, 
                                size=array.nbytes))
        for array in output_arrays]

In [14]:
def cleanup_memories(tuples):
    for (_, buffer) in tuples:
        buffer.release()

In [15]:
a = numpy.random.rand(N).astype(numpy.float32)
b = numpy.random.rand(N).astype(numpy.float32)
c = numpy.empty_like(a)

# Device Memory setup
input_tuples = create_input_memory(nvidia_context, (a,b,))
output_tuples = create_output_memory(nvidia_context, (c,),)
run_ocl_kernel(nvidia_queue, nvidia_program.sum, (N,), input_tuples, output_tuples)
check_sum_results(a,b,c)

result matches!


In [16]:
def build_program(ocl_file, context, build_options=[]):
    with open(ocl_file,"r") as opencl_file:
        program_source_code = opencl_file.read()
        program_source = pyopencl.Program(context, program_source_code)
        program = program_source.build(options=build_options)
        
        return program

In [17]:
batch_size = 16
wg_size = 32
power = 1
build_options = ["-D","BATCH_SIZE={}".format(batch_size),
                 "-D","WG_SIZE={}".format(wg_size),
                 "-D","POW={}".format(power)]

nvidia_program = build_program("vector_sum.cl", nvidia_context, build_options)
run_ocl_kernel(nvidia_queue, nvidia_program.sum, (N,), input_tuples, output_tuples, (wg_size,))
check_sum_results(a,b,c)

result matches!


## Batching

```c
kernel void sum_batched(global float *a, 
                        global float *b,
                        global float *c){
  int gid = get_global_id(0)*BATCH_SIZE;
    
  for(int i=0; i<BATCH_SIZE;++i)
     c[gid + i] = a[gid + i] + b[gid + i];
}
```

In [18]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum_batched, (N//batch_size,), input_tuples, output_tuples)

3.62 ms ± 543 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Using local and private memory

## Batched + Private Memory

```c
kernel void sum_batched_private(global float *a, 
                                global float *b,
                                global float *c){
  int gid = get_global_id(0)*BATCH_SIZE;
    
  float a_tmp[BATCH_SIZE], b_tmp[BATCH_SIZE], c_tmp[BATCH_SIZE];
      
  for(int i=0; i<BATCH_SIZE;++i){
    a_tmp[i] = a[gid + i];
    b_tmp[i] = b[gid + i];
  }
    
  for(int i=0; i<BATCH_SIZE;++i)
    c_tmp[i] = a_tmp[i] + b_tmp[i];
    
  for(int i=0; i<BATCH_SIZE;++i) 
    c[gid + i] = c_tmp[i];
}
```

In [19]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum_batched_private, (N//batch_size,), input_tuples, output_tuples)

2.53 ms ± 549 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# How to Do Things in Parallel

In this section:
* Inspecting the characteristics of the available devices
* Use the work-items and work-groups abstraction to chose between task and data parallelism.

![Cat yarn gif](./img/cat_yarn.gif)

## Device characteristics
Using the host code API to inspect the characteristics of the device.

In [20]:
name_properties = {
    "Device Name": pyopencl.device_info.NAME,
    "Device Platform": pyopencl.device_info.PLATFORM,
    "Device Type": pyopencl.device_info.TYPE
}

processing_properties = {
    "Available Compute Units": pyopencl.device_info.MAX_COMPUTE_UNITS,
    "Clockrate": pyopencl.device_info.MAX_CLOCK_FREQUENCY
}

memory_properties = {
    "Available Global Memory": pyopencl.device_info.GLOBAL_MEM_SIZE,
    "Available Constant Memory": pyopencl.device_info.MAX_CONSTANT_BUFFER_SIZE,
    "Available Local Memory" : pyopencl.device_info.LOCAL_MEM_SIZE
}

device_types = {
    pyopencl.device_type.CPU: "CPU",
    pyopencl.device_type.GPU: "GPU"
}

In [21]:
intel_platform = [platform for platform in pyopencl.get_platforms() 
                  if platform.name == "Intel(R) OpenCL"][0]

In [22]:
for device in intel_platform.get_devices():
    #print out all of the device name properties, except the device type
    for property_name in sorted(name_properties.keys() - {"Device Type"}):
        property_string_args = (property_name,device.get_info(name_properties[property_name]))
        print("{}: {}".format(*property_string_args))
        
    #look up the device type
    print("Device Types: {}".format(device_types[device.get_info(name_properties["Device Type"])]))
    
    #print out all of the processing properties
    for property_name in sorted(processing_properties.keys()):
        property_string_args = (property_name,device.get_info(processing_properties[property_name]))
        print("{}: {}".format(*property_string_args))
    
    #print out all of the memory properties
    for property_name in sorted(memory_properties.keys()):
        property_string_args = (property_name,device.get_info(memory_properties[property_name]))
        print("{}: {}".format(*property_string_args))
        
    print("\n")

Device Name: Intel(R) HD Graphics
Device Platform: <pyopencl.Platform 'Intel(R) OpenCL' at 0x321d400>
Device Types: GPU
Available Compute Units: 23
Clockrate: 1100
Available Constant Memory: 4294959103
Available Global Memory: 13321633792
Available Local Memory: 65536


Device Name: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz
Device Platform: <pyopencl.Platform 'Intel(R) OpenCL' at 0x321d400>
Device Types: CPU
Available Compute Units: 8
Clockrate: 2800
Available Constant Memory: 131072
Available Global Memory: 16662528000
Available Local Memory: 32768




In [23]:
!clinfo

Number of platforms                               3
  Platform Name                                   Intel(R) OpenCL
  Platform Vendor                                 Intel(R) Corporation
  Platform Version                                OpenCL 2.0 
  Platform Profile                                FULL_PROFILE
  Platform Extensions                             cl_khr_3d_image_writes cl_khr_byte_addressable_store cl_khr_depth_images cl_khr_fp64 cl_khr_global_int32_base_atomics cl_khr_global_int32_extended_atomics cl_khr_icd cl_khr_image2d_from_buffer cl_khr_local_int32_base_atomics cl_khr_local_int32_extended_atomics cl_khr_spir
  Platform Extensions function suffix             INTEL

  Platform Name                                   Portable Computing Language
  Platform Vendor                                 The pocl project
  Platform Version                                OpenCL 1.2 pocl 1.1 None+Asserts, LLVM 6.0.0, SPIR, SLEEF, DISTRO, POCL_DEBUG
  Platform Profile               

  Preferred work group size multiple              128
  Preferred / native vector sizes                 
    char                                                 1 / 32      
    short                                                1 / 16      
    int                                                  1 / 8       
    long                                                 1 / 4       
    half                                                 0 / 0        (n/a)
    float                                                1 / 8       
    double                                               1 / 4        (cl_khr_fp64)
  Half-precision Floating-point support           (n/a)
  Single-precision Floating-point support         (core)
    Denormals                                     Yes
    Infinity and NANs                             Yes
    Round to nearest                              Yes
    Round to zero                                 No
    Round to infinity                             No
    

  Preferred work group size multiple              32
  Warp size (NV)                                  32
  Preferred / native vector sizes                 
    char                                                 1 / 1       
    short                                                1 / 1       
    int                                                  1 / 1       
    long                                                 1 / 1       
    half                                                 0 / 0        (n/a)
    float                                                1 / 1       
    double                                               1 / 1        (cl_khr_fp64)
  Half-precision Floating-point support           (n/a)
  Single-precision Floating-point support         (core)
    Denormals                                     Yes
    Infinity and NANs                             Yes
    Round to nearest                              Yes
    Round to zero                                 Yes
    

## Workitems vs Workgroups
Exploring different types of parallelism, with our trusty vector sum.

## OpenCL Task vs Data Parallelism Abstractions

![OpenCL Task vs Data Abstractions](./img/opencl_task_data_parallelism.png)

## Vectorising

```c
kernel void sum16(global float16 *a, 
                  global float16 *b,
                  global float16 *c){
  int gid = get_global_id(0);
    
  c[gid] = a[gid] + b[gid];
}
```

In [24]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum16, (N//16,), input_tuples, output_tuples)

2.36 ms ± 528 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Vectorised + Local Memory

```c
kernel void sum16_local(global float16 *a, 
                        global float16 *b,
                        global float16 *c){
  int wid = get_group_id(0)*WG_SIZE;
  int lid = get_local_id(0);
    
  local float16 a_local[WG_SIZE], b_local[WG_SIZE], c_local[WG_SIZE];
    
  // Copying on
  event_t copyon[2];
  copyon[0] = async_work_group_copy(a_local, a + wid, WG_SIZE, 0);
  copyon[1] = async_work_group_copy(b_local, b + wid, WG_SIZE, 0);
  wait_group_events(2, copyon);
    
  c_local[lid] = a_local[lid] + b_local[lid];
    
  // Copying off
  event_t copyoff[1];
  copyoff[0] = async_work_group_copy(c + wid, c_local, WG_SIZE, 0);
  wait_group_events(1, copyoff);
}
```

In [30]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum16_local, (N//16,), input_tuples, output_tuples)

1.97 ms ± 51.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## But, before we think that we're too clever:

What about good old `numpy`?

In [26]:
%timeit (a+b)

668 µs ± 157 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Increasing the computational work

Now we make the kernel: $\vec{c} = {(\vec{a} + \vec{b}) }^{x}$

```c
kernel void sum16_pow(global float16 *a, 
                      global float16 *b,
                      global float16 *c){
  int gid = get_global_id(0);
    
  c[gid] = pow(a[gid] + b[gid], POW);
}
```

Make $x=1000$

In [27]:
power = 1000
build_options = ["-D","BATCH_SIZE={}".format(batch_size),
                 "-D","WG_SIZE={}".format(wg_size),
                 "-D","POW={}".format(power)]

nvidia_program = build_program("vector_sum.cl", nvidia_context, build_options)

In [28]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum16_pow, (N//16,), input_tuples, output_tuples)

3.26 ms ± 538 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%timeit (a+b)**power

/home/gordon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in power
  """Entry point for launching an IPython kernel.


24.9 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Ca(t)veats

![Cat no gif](./img/cat_no.gif)

![Amdahl's law](./img/amdahl_graph.png)

Source <a href="https://en.wikipedia.org/wiki/User:Daniels220" class="extiw" title="wikipedia:User:Daniels220">Daniels220</a> at <a href="https://en.wikipedia.org/wiki/" class="extiw" title="wikipedia:">English Wikipedia</a>, <a href="https://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=6678551">Link</a>

![XKCD is it worth the time?](./img/xkcd_is_it_worth_the_time.png)

# Thank you!

![Cat thank you](./img/cat_thankyou.gif)

?

# Recommended Reading List
* Gene Amdahl, Validity of the Single Processor Approach to Achieving Large-Scale Computing Capabilities
* Page and Luk, Compiling Occam into field-programmable gate arrays
* Herb Sutter, The Free Lunch is Over
* Asanovic et al., The Landscape of Parallel Computing Research: A View from Berkeley
* Tsugio Makimoto, The Hot Decade of Field Programmable Technologies
* Lee et al., Debunking the 100X GPU vs CPU myth
* Che et al., Rodinia: A Benchmark Suite for Heterogeneous Computing
* Thomas et al., Hardware architectures for Monte-Carlo based financial simulations
* Mike Giles, Some (strong) opinions on HPC and the use of GPUs